# 1. Title

# Implementing AdaBoost for Predicting Online Shoppers' Purchase Intentions

- Team: 404 Not Found

- Team Members: Diksha Krishnan, Xinyu Zhou, Shen Yu, Dongyan Sun 

- Date: December 9, 2024

# 2. Overview of AdaBoost

# 2.1. An overview of the algorithm and describe its advantages and disadvantages.

- AdaBoost, short for Adaptive Boosting, is a powerful ensemble learning algorithm that improves the accuracy of weak learners by combining them into a strong model. It works by iteratively training weak learners, such as decision stumps, and assigning higher weights to samples that are misclassified. 

- AdaBoost offers significant advantages but also comes with certain disadvantages. Its simplicity and flexibility allow it to be implemented easily with various weak learners, and it works effectively for both classification and regression problems. By focusing on misclassified samples, AdaBoost transforms weak learners that perform only slightly better than random guessing into a strong ensemble model. AdaBoost is also robust to overfitting when properly tuned and can handle non-linear problems by combining weak learners to form complex decision boundaries.

- One of its disadvantages is that AdaBoost is sensitive to outliers because misclassified samples are given higher weights, which can lead to poor performance if the misclassified samples are noisy or outliers. The algorithm requires careful tuning of hyperparameters, such as the number of iterations and learning rate, to achieve optimal performance without overfitting or underfitting. The effectiveness of AdaBoost also depends on the quality of the weak learners. If the weak learners are too complex, the model may overfit the training data. Additionally, AdaBoost can be computationally intensive due to its iterative nature, particularly when applied to large datasets.

# 2.2. Representation: describe how the feature values are converted into a single number prediction.

In this case of the project, the weak leaner is Disicion Stumps. AdaBoost creates a weighted combination of these stumps, resulting in a more complex decision boundary that better separates the data.

Each weak learner is assigned a weight ${w_m}$, which reflects its accuracy. The weight ${w_m}$ is calculated as:


$${w_m} = \frac{1}{2} \ln \left( \frac{1 - \epsilon_m}{\epsilon_m} \right)$$


where $\epsilon_m$ is the weighted error rate of the weak learner. A smaller error leads to a larger weight, indicating a better learner.

The final prediction $H(x)$ of the AdaBoost model is a weighted sum of the predictions from all weak learners:


$$H(x) = \text{sign} \left( \sum_{m=1}^M {w_m} h_m(x) \right)$$



$M$: Total number of weak learners.

$h_m(x)$: Prediction of the $m$-th weak learner (usually $+1$ or $-1$ for binary classification).

${w_m}$: Weight of the $m$-th weak learner.

$\text{sign}$: Ensures the final output is a classification decision (e.g., $+1$ or $-1$).

# 2.3. Loss: describe the metric used to measure the difference between the model’s prediction and the target variable.

AdaBoost minimizes an exponential loss function which ensures that large errors (misclassified points) contribute more significantly to the loss, encouraging the model to focus on correcting them.

$$\mathcal{L}(H) = \sum_{i=1}^m \exp(-y_i H(x_i))$$

# 2.4. Optimizer: describe the numerical algorithm used to find the model parameters that minimize the loss given a training set.

# 2.5. Pseudo Code to explain how numerical algorithms work.

$\text{Input: } $

$\quad S = \{(x_1, y_1), \dots, (x_m, y_m)\}, \text{ where } y_i \in \{-1, +1\},$

$\quad\text{ number of iterations } T,$

$\quad\text{ weak learner } WL$

$\text{Initialize: Sample weights } D_i^{(1)} = \frac{1}{m}, \forall i = 1, \dots, m.$

$\quad \textbf{for } t = 1 \textbf{ to } T:$

$\quad\quad \text{Invoke weak learner } h_t=WL(D^{(t)},S) $

$\quad\quad \text{Compute error rate: } \epsilon_t = \sum_{i=1}^m D_i^{(t)} \mathbb{1}[h_t(x_i) \neq y_i].$

$\quad\quad \text{Let } w_t = \frac{1}{2} \ln\left(\frac{1 - \epsilon_t}{\epsilon_t}\right).$

$\quad\quad \text{Update sample weights: } D_i^{(t+1)} = \frac{D_i^{(t)} \exp(-w_t y_i h_t(x_i))}{Z_t},$

$\quad\quad \text{where } Z_t = \sum_{i=1}^m D_i^{(t)} \exp(-w_t y_i h_t(x_i))\text{, for all }i=1,...,m$

$\text{Output: Final hypothesis: } h_s(x) = \text{sign}\left(\sum_{t=1}^T w_t h_t(x)\right).$

# 2.6. References and Citations 

- Shalev-Shwartz, S. and Ben-David, S. (2014) Understanding Machine Learning: From Theory to Algorithms. Cambridge: Cambridge University Press.

- In pages 105-112 of the book, the mathematics and numerical algorithms behind AdaBoost, along with its application in face recognition, are discussed. Learned from both the book and real practice, AdaBoost relies heavily on numerical techniques such as exponentiation, logarithms, and weight normalization to ensure the algorithm’s stability and efficiency during training. One key technique is exponentiation for weight updates, where AdaBoost increases the weights of misclassified samples exponentially based on the weak classifier's error. This allows the algorithm to focus more on challenging samples in subsequent rounds. However, exponentiation can lead to very large or small values, which could cause numerical instability. To prevent issues like underflow or overflow, AdaBoost uses efficient computation of exponential functions to maintain stability across a range of values. Another important numerical technique is the logarithmic computation of alpha, which adjusts the weight given to each weak classifier based on its performance. Weak classifiers with low error rates receive higher alpha values, contributing more to the final prediction. Using logarithms to compute alpha ensures that the values remain within a manageable range, preventing excessively large or small values that could destabilize training. Finally, normalization of weights is applied after each update to prevent the weights from growing too large. This ensures that the total weight remains constant, allowing AdaBoost to continue focusing on misclassified samples without causing numerical instability. These techniques are essential for AdaBoost's reliable performance, especially on complex real-world datasets.

# 3. Model

In [1]:
# Add docstrings to each method and function and explain what they do and what the inputs and outputs are
import numpy as np
import pandas as pd

class AdaBoost:
    def __init__(self, n_estimators=50):
        """
        Initialize the AdaBoost model.
        Args:
            n_estimators (int): The number of weak classifiers to use in the ensemble.
        Attributes:
            alphas (list): Weights of each weak classifier.
            models (list): Parameters of the weak classifiers (decision stumps).
        """
        self.n_estimators = n_estimators
        self.alphas = []  # Store the weights of the weak learners
        self.models = []  # Store weak classifiers (decision stumps)

    def _build_stump(self, X, y, weights):
        """
        Build a decision stump, which is a weak classifier based on a single feature and threshold.
        Args:
            X (numpy.ndarray): Feature matrix of shape (n_samples, n_features).
            y (numpy.ndarray): Target labels of shape (n_samples,).
            weights (numpy.ndarray): Weights for each sample of shape (n_samples,).

        Returns:
            stump (dict): Parameters of the best decision stump.
            min_error (float): Minimum classification error achieved by the stump.
        """
        n_samples, n_features = X.shape
        min_error = float("inf")  # Initialize with a high error
        stump = {}  # To store the best stump parameters

        for feature in range(n_features):
            # Iterate over all unique thresholds for the feature
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                for polarity in [-1, 1]:
                    # Predict using the current threshold and polarity
                    predictions = np.ones(n_samples)
                    if polarity == 1:
                        predictions[X[:, feature] <= threshold] = -1
                    else:
                        predictions[X[:, feature] > threshold] = -1

                    # Calculate weighted error
                    error = np.sum(weights[predictions != y])
                    if error < min_error:
                        # Update the best stump parameters if error is minimized
                        min_error = error
                        stump['feature'] = feature
                        stump['threshold'] = threshold
                        stump['polarity'] = polarity
                        stump['predictions'] = predictions

        return stump, min_error

    def train(self, X, y):
        """
        Train the AdaBoost classifier using the training data.
        Args:
            X (numpy.ndarray): Feature matrix of shape (n_samples, n_features).
            y (numpy.ndarray): Target labels of shape (n_samples,).

        Modifies:
            self.models: Appends the parameters of each weak learner.
            self.alphas: Appends the weight of each weak learner.
        """
        n_samples, _ = X.shape
        weights = np.ones(n_samples) / n_samples  # Initialize uniform sample weights

        for _ in range(self.n_estimators):
            # Build the best weak classifier
            stump, error = self._build_stump(X, y, weights)
            # Compute the alpha (importance) of the stump
            alpha = 0.5 * np.log((1 - error) / (error + 1e-10))
            stump['alpha'] = alpha

            # Update sample weights
            weights *= np.exp(-alpha * y * stump['predictions'])
            weights /= np.sum(weights)  # Normalize weights

            # Store the stump and its weight
            self.models.append(stump)
            self.alphas.append(alpha)

    def predict(self, X):
        """
        Predict the class labels for the given data using the trained AdaBoost model.
        Args:
            X (numpy.ndarray): Feature matrix of shape (n_samples, n_features).

        Returns:
            numpy.ndarray: Predicted labels of shape (n_samples,), values in {1, -1}.
        """
        n_samples = X.shape[0]
        final_prediction = np.zeros(n_samples)  # Accumulate weighted predictions

        for model in self.models:
            # Extract parameters of the weak classifier
            feature = model['feature']
            threshold = model['threshold']
            polarity = model['polarity']
            alpha = model['alpha']

            # Make predictions using the weak classifier
            predictions = np.ones(n_samples)
            if polarity == 1:
                predictions[X[:, feature] <= threshold] = -1
            else:
                predictions[X[:, feature] > threshold] = -1

            # Add weighted predictions to final prediction
            final_prediction += alpha * predictions

        return np.sign(final_prediction)  # Return the final ensemble prediction

    @staticmethod
    def f1_score(y_true, y_pred):
        """
        Compute the F1 Score of the predictions.
        Args:
            y_true (numpy.ndarray): True labels of shape (n_samples,).
            y_pred (numpy.ndarray): Predicted labels of shape (n_samples,).

        Returns:
            float: F1 Score.
        """
        tp = np.sum((y_true == 1) & (y_pred == 1))
        fp = np.sum((y_true == -1) & (y_pred == 1))
        fn = np.sum((y_true == 1) & (y_pred == -1))

        precision = tp / (tp + fp + 1e-10)
        recall = tp / (tp + fn + 1e-10)
        return 2 * (precision * recall) / (precision + recall + 1e-10)

    @staticmethod
    def accuracy(y_true, y_pred):
        """
        Compute the accuracy of the predictions.
        Args:
            y_true (numpy.ndarray): True labels of shape (n_samples,).
            y_pred (numpy.ndarray): Predicted labels of shape (n_samples,).

        Returns:
            float: Accuracy.
        """
        return np.mean(y_true == y_pred)


# 3. Model - Main

In [ ]:
def preprocess_data(file_path):
    """
    Read and preprocess the dataset from a CSV file.
    Args:
        file_path (str): Path to the CSV file.

    Returns:
        X (numpy.ndarray): Preprocessed feature matrix.
        y (numpy.ndarray): Target labels converted to {1, -1}.
    """
    df = pd.read_csv(file_path)
    
    # Encode categorical features using one-hot encoding
    categorical_features = ['OperatingSystems','Month', 'Browser', 'Region', 'TrafficType', 'VisitorType', 'Weekend']
    df = pd.get_dummies(df, columns=categorical_features, drop_first=True)

    # Combine all numerical features for standard scaling
    numerical_features = [
        'Administrative', 'Informational', 'Administrative_Duration',
        'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration', 'BounceRates',
        'ExitRates', 'PageValues', 'SpecialDay'
    ]

    # Standard scale numerical features
    for col in numerical_features:
        df[col] = (df[col] - df[col].mean()) / (df[col].std() + 1e-10)

    # Separate features (X) and labels (y)
    X = df.drop(columns=['Revenue']).values  # Feature matrix
    y = df['Revenue'].values                 # Target labels
    y = np.where(y == 1, 1, -1)              # Convert labels to {1, -1}

    return X, y



def stratified_split(X, y):
    """
    Split the dataset into training, validation, and test sets while maintaining class ratios.
    Args:
        X (numpy.ndarray): Feature matrix of shape (n_samples, n_features).
        y (numpy.ndarray): Target labels of shape (n_samples,).

    Returns:
        Tuple[numpy.ndarray]: (X_train, y_train, X_val, y_val, X_test, y_test).
    """
    n_samples = len(y)
    neg_indices = np.where(y == -1)[0]
    pos_indices = np.where(y == 1)[0]

    # Shuffle indices
    np.random.shuffle(neg_indices)
    np.random.shuffle(pos_indices)

    # Split indices by class
    neg_train_end = int(0.6 * len(neg_indices))
    neg_val_end = int(0.8 * len(neg_indices))
    pos_train_end = int(0.6 * len(pos_indices))
    pos_val_end = int(0.8 * len(pos_indices))

    train_indices = np.concatenate([neg_indices[:neg_train_end], pos_indices[:pos_train_end]])
    val_indices = np.concatenate([neg_indices[neg_train_end:neg_val_end], pos_indices[pos_train_end:pos_val_end]])
    test_indices = np.concatenate([neg_indices[neg_val_end:], pos_indices[pos_val_end:]])

    np.random.shuffle(train_indices)
    np.random.shuffle(val_indices)
    np.random.shuffle(test_indices)

    return X[train_indices], y[train_indices], X[val_indices], y[val_indices], X[test_indices], y[test_indices]


def main():
    # Load and preprocess the dataset
    file_path = '/Users/emmasun/Desktop/2060/project/online_shoppers_intention.csv'
    X, y = preprocess_data(file_path)

    # Perform stratified split
    X_train, y_train, X_val, y_val, X_test, y_test = stratified_split(X, y)

    # Train the AdaBoost model
    model = AdaBoost(n_estimators=50)
    model.train(X_train, y_train)

    # Evaluate the model
    for split_name, (X_split, y_split) in zip(
        ['Training', 'Validation', 'Test'],
        [(X_train, y_train), (X_val, y_val), (X_test, y_test)]
    ):
        predictions = model.predict(X_split)
        acc = model.accuracy(y_split, predictions)
        f1 = model.f1_score(y_split, predictions)
        print(f"{split_name} Accuracy: {acc:.4f}, F1 Score: {f1:.4f}")


if __name__ == "__main__":
    main()

# 4. Check Model

# 4.1. Unit Test

# 4.2. Previous work where AdaBoost is applied on a public dataset. 

- Swetha, T., R, R., Sajitha, T., B, V., Sravani, J. and Praveen, B. (2024) 'Forecasting Online Shoppers Purchase Intentions with Cat Boost Classifier', 2024 International Conference on Distributed Computing and Optimization Techniques (ICDCOT), Bengaluru, India, 2024, pp. 1-6. doi: 10.1109/ICDCOT61034.2024.10515309.

- In the previous study, a system was proposed to predict online shoppers' intentions (whether they will buy or not) by analyzing various classification algorithms, including Random Forest, Decision Tree, Support Vector Machine, Gradient Boosting, AdaBoost, LightGBM, and CatBoost. The experiments were conducted using an online shoppers' intention dataset. Hyperparameter tuning was performed by first selecting an appropriate assessment metric, such as accuracy or ROC-AUC, to evaluate the model's performance. Practitioners then chose a search method, such as grid search or random search, to effectively explore the hyperparameter space. The search space for each hyperparameter was defined by a range of values or distributions. Cross-validation was employed to assess the model's performance across different hyperparameter configurations while preventing overfitting. The optimal hyperparameter values were determined by splitting the dataset into training and validation sets and iteratively training the model with different configurations. The final performance of the tuned model was evaluated on a separate test set to ensure generalization to unseen data. Experimental results showed that AdaBoost, with hyperparameter tuning, achieved an accuracy of 89.14%, while CatBoost, also with hyperparameter tuning, delivered the highest accuracy of 98.73%.

- In our work, we further explore the impact of different numbers of estimators on the performance of the AdaBoost model. Specifically, we test 9 different values for n_estimators — [10, 50, 100, 150, 200, 250, 300, 350, 400] — and observe similar results. Our work achieved accuracy of 90.56% compared to 89.14%, F1 score of 66.18% compared to 63.26%, precision of 74.27% compared to 69.34%, and recall of 59.69% compared to 58.17%. This investigation helps validate the robustness of the AdaBoost model and its performance across varying configurations.

In [ ]:
from sklearn.metrics import precision_score, recall_score

def main():
    # Load and preprocess the dataset
    file_path = '/Users/emmasun/Desktop/2060/project/online_shoppers_intention.csv'
    X, y = preprocess_data(file_path)

    # Perform stratified split
    X_train, y_train, X_val, y_val, X_test, y_test = stratified_split(X, y)

    # Define the list of n_estimators to try
    n_estimators_list = [10, 50, 100, 150, 200, 250, 300, 350, 400]

    # Iterate over different values of n_estimators
    for n_estimators in n_estimators_list:
        # Initialize and train the AdaBoost model
        model = AdaBoost(n_estimators=n_estimators)
        model.train(X_train, y_train)

        # Evaluate the model on the test set only
        predictions = model.predict(X_test)

        # Compute the accuracy, F1 score, precision, and recall
        acc = model.accuracy(y_test, predictions)
        f1 = model.f1_score(y_test, predictions)
        precision = precision_score(y_test, predictions)
        recall = recall_score(y_test, predictions)

        # Print the evaluation results for the test set
        print(f"Test Set Evaluation for {n_estimators} Estimators:")
        print(f"Accuracy: {acc:.4f}, F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
        print("-" * 50)

if __name__ == "__main__":
    main()

# 5. Github repo

- https://github.com/delio05/AdaBoost

# 6. References

- Shalev-Shwartz, S. and Ben-David, S. (2014) Understanding Machine Learning: From Theory to Algorithms. Cambridge: Cambridge University Press.

- Swetha, T., R, R., Sajitha, T., B, V., Sravani, J. and Praveen, B. (2024) 'Forecasting Online Shoppers Purchase Intentions with Cat Boost Classifier', 2024 International Conference on Distributed Computing and Optimization Techniques (ICDCOT), Bengaluru, India, 2024, pp. 1-6. doi: 10.1109/ICDCOT61034.2024.10515309.

- Sakar, C. & Kastro, Y. (2018). Online Shoppers Purchasing Intention Dataset [Dataset]. UCI Machine Learning Repository. https://doi.org/10.24432/C5F88Q.


# 7. Contributions

- Thank you to Diksha Krishnan, Xinyu Zhou, Shen Yu, and Dongyan Sun for their equal contributions to the final project.